# Text Analytics Group Assignment2
Due: September 25, 2017, by 11:59 p.m. In this assignment you are an analytics consultant to a   (i) brand manager, (ii) product manager and (iii)advertising manager. Your job is    to give advice/insights to these individuals based on the analysis of social media conversations. The detailed tasks are described below. We use cars as an example of a “high involvement” good (recall from class discussions that for high involvement goods, people use social media heavily for awareness building and research).

1.Develop a   crawler/scraper to fetch messages posted in Edmunds.com discussion forums. The crawler output should be a .csv file with the following columns: date, userid, and message (even though you will only use the messages in your analysis). Before you develop the crawler, carefully study one of the forums on Edmunds.com to understand the html as well as the threading structures.

2.Fetch between 5,000 and 10,000 posts about cars from a General topics forum. Do NOT choosea forum dedicated to a   particular brand or model. Instead, you can choose the General & Sedans categories and then select, for example, the Entry Level Luxury forum https://forums.edmunds.com/discussion/2864/general/x/entry-level-luxury-performance-sedansThe idea is    to have multiple brands and models being discussed without one of them being the focal point.

3.Once you fetch the data, find the top 10 brands from frequency counts. You will need to write a script to count the frequencies. Be sure not to count a mention more than once per post, evenif   it    is mentioned multiple times.Replace modelswith brands so that from now on you have to deal with only brands and not models. You will need another script for this job. This step is meant to help simplify the analysis. A    list of model and brand names (not exhaustive) are provided in a separate file.

#### Task A:
Identify top 10 brands by frequency. From the posts, calculate lift ratios for associations between the brands. You will have to write a   script to do this task). Show the brands on a   multi-dimensional scaling (MDS) map (use a Python script for MDS, there are multiple scripts available onthe Internet). 

In [45]:
import pandas as pd
from nltk.corpus import stopwords

In [46]:
#car forum data
forum_data =pd.read_csv('reed_new_out.csv', header = 0)

df2=pd.DataFrame([['April 12, 2002 2:53PM','dani','bmw toyota pontiac honda acura audi nissan infiniti ford subaru']], columns=['date','userid','message'])
forum_data=forum_data.append(df2)
forum_data['message']=forum_data['message'].str.lower()
forum_data[:3]

,Unnamed: 0,date,message,userid
0,0.0,"March 23, 2002 7:04PM",entry level performance luxury sedans are a ho...,cybersol
1,1.0,"March 25, 2002 5:54AM",i personally think that with a few tweaks the ...,merc1
2,2.0,"March 25, 2002 7:06AM",i am debating a new purchase and these two are...,fredvh


In [47]:
#list of car brands/names
model_list =pd.read_csv('Cars make model list3.csv', header = 2)
model_list['Search']=model_list['Search'].str.lower()

In [48]:
##some of the Search values from the model_list df have a weird format after that we need to remove
#'legend\xca'.replace('\xca','')
model_list['Search'].replace('\xca','').values
ls=model_list['Search'].tolist()

for i in range(len(ls)):
    ls[i]=ls[i].replace('\xca','')

ls                              #the correct names now! add this back to model_list df
model_list['Search1']=ls        #setting new column equal to correct values

In [49]:
def replace_model(s):
    for model in model_list['Search1'].values:
        if model in s:
            s=s.replace(model,model_list[model_list['Search1']==model]['Replace'].values[0])
    return s

In [50]:
#mapping that function to forum df
forum_data['message_clean']=forum_data['message'].astype(str).map(replace_model)
forum_data[:3]

,Unnamed: 0,date,message,userid,message_clean
0,0.0,"March 23, 2002 7:04PM",entry level performance luxury sedans are a ho...,cybersol,entry level performance luxury sedans are a ho...
1,1.0,"March 25, 2002 5:54AM",i personally think that with a few tweaks the ...,merc1,i personally think that with a few tweaks the ...
2,2.0,"March 25, 2002 7:06AM",i am debating a new purchase and these two are...,fredvh,i am debating a new purchase and these two are...


In [51]:
#removing stopwords and adding to new column in df
stop = stopwords.words('english')
forum_data['message_without_stopwords'] = forum_data['message_clean'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop)]))

In [52]:
#count the number of messages that contain each brand name
#careful- don't want to count same message twice if brand is stated twice

counter={}
for brand in model_list['Replace'].drop_duplicates():
    if len(forum_data['message_without_stopwords'].str.contains(brand).value_counts().values)>1:
        count=forum_data['message_without_stopwords'].str.contains(brand).value_counts()[True]
    else:
        count=0
    counter[brand]=count
counter

{'acura': 675,
 'audi': 594,
 'bmw': 1866,
 'buick': 52,
 'cadillac': 133,
 'chevrolet': 173,
 'chrysler': 101,
 'dodge': 241,
 'ford': 296,
 'honda': 683,
 'hyundai': 247,
 'infiniti': 506,
 'kia': 29,
 'lincoln': 74,
 'mazda': 118,
 'mercedes': 265,
 'mercury': 8,
 'mitsubishi': 27,
 'nissan': 562,
 'pontiac': 1088,
 'saturn': 33,
 'subaru': 277,
 'suzuki': 25,
 'toyota': 1654,
 'volkswagen': 236,
 'volvo': 156}

In [53]:
counterlist=[] #list of [brand, #messages brand appears in]
for key, value in counter.iteritems():
    temp = [key,value]
    counterlist.append(temp)

top10=sorted(counterlist, key=lambda x: x[1],reverse=True)[:10]
top10 #list of top 10 [brand, #messages] 

[['bmw', 1866],
 ['toyota', 1654],
 ['pontiac', 1088],
 ['honda', 683],
 ['acura', 675],
 ['audi', 594],
 ['nissan', 562],
 ['infiniti', 506],
 ['ford', 296],
 ['subaru', 277]]

In [54]:
#dictionary of top 10 to values and list of top 10 brands
num={}  #dictionary of top 10 {'brand':#messages}
for i in top10:
    num[i[0]]=i[1]
num

listof10=[]   
for i in top10:
    listof10.append(i[0])
listof10   #list of only brand names

['bmw',
 'toyota',
 'pontiac',
 'honda',
 'acura',
 'audi',
 'nissan',
 'infiniti',
 'ford',
 'subaru']

In [55]:
#getting combinations of top 10 brand names, naming the list of (brand1, brand2) as comb
import itertools
print list(itertools.combinations(listof10,2))
comb=list(itertools.combinations(listof10,2))

[('bmw', 'toyota'), ('bmw', 'pontiac'), ('bmw', 'honda'), ('bmw', 'acura'), ('bmw', 'audi'), ('bmw', 'nissan'), ('bmw', 'infiniti'), ('bmw', 'ford'), ('bmw', 'subaru'), ('toyota', 'pontiac'), ('toyota', 'honda'), ('toyota', 'acura'), ('toyota', 'audi'), ('toyota', 'nissan'), ('toyota', 'infiniti'), ('toyota', 'ford'), ('toyota', 'subaru'), ('pontiac', 'honda'), ('pontiac', 'acura'), ('pontiac', 'audi'), ('pontiac', 'nissan'), ('pontiac', 'infiniti'), ('pontiac', 'ford'), ('pontiac', 'subaru'), ('honda', 'acura'), ('honda', 'audi'), ('honda', 'nissan'), ('honda', 'infiniti'), ('honda', 'ford'), ('honda', 'subaru'), ('acura', 'audi'), ('acura', 'nissan'), ('acura', 'infiniti'), ('acura', 'ford'), ('acura', 'subaru'), ('audi', 'nissan'), ('audi', 'infiniti'), ('audi', 'ford'), ('audi', 'subaru'), ('nissan', 'infiniti'), ('nissan', 'ford'), ('nissan', 'subaru'), ('infiniti', 'ford'), ('infiniti', 'subaru'), ('ford', 'subaru')]


In [56]:
#counting how many messages contain each combination of brands
#counter_comb={(brand1, brand2):num of messages}

counter_comb={}
for brand in comb:
    mask_b0=(forum_data['message_without_stopwords'].str.contains(brand[0]))
    mask_b1=(forum_data['message_without_stopwords'].str.contains(brand[1]))
    mask=(mask_b0 & mask_b1)
    count=len(forum_data['message_without_stopwords'][mask])
    counter_comb[brand]=count
counter_comb

{('acura', 'audi'): 148,
 ('acura', 'ford'): 57,
 ('acura', 'infiniti'): 178,
 ('acura', 'nissan'): 123,
 ('acura', 'subaru'): 61,
 ('audi', 'ford'): 55,
 ('audi', 'infiniti'): 132,
 ('audi', 'nissan'): 86,
 ('audi', 'subaru'): 51,
 ('bmw', 'acura'): 310,
 ('bmw', 'audi'): 316,
 ('bmw', 'ford'): 137,
 ('bmw', 'honda'): 279,
 ('bmw', 'infiniti'): 310,
 ('bmw', 'nissan'): 216,
 ('bmw', 'pontiac'): 499,
 ('bmw', 'subaru'): 106,
 ('bmw', 'toyota'): 687,
 ('ford', 'subaru'): 26,
 ('honda', 'acura'): 193,
 ('honda', 'audi'): 95,
 ('honda', 'ford'): 72,
 ('honda', 'infiniti'): 87,
 ('honda', 'nissan'): 161,
 ('honda', 'subaru'): 45,
 ('infiniti', 'ford'): 40,
 ('infiniti', 'subaru'): 44,
 ('nissan', 'ford'): 57,
 ('nissan', 'infiniti'): 108,
 ('nissan', 'subaru'): 41,
 ('pontiac', 'acura'): 210,
 ('pontiac', 'audi'): 171,
 ('pontiac', 'ford'): 59,
 ('pontiac', 'honda'): 189,
 ('pontiac', 'infiniti'): 267,
 ('pontiac', 'nissan'): 181,
 ('pontiac', 'subaru'): 72,
 ('toyota', 'acura'): 269,
 ('t

In [57]:
#calculate lift for associations between the top 10 brands
#lift=number of messages*num messages with a and b / (num messages with a) * (num messages with b)

def lift(a,b):
    num=len(forum_data)
    num_a_and_b=counter_comb[(a,b)]
    num_a=float(counter[a])
    num_b=float(counter[b])
    lift_val=(num*num_a_and_b)/(num_a*num_b)
    return lift_val

lift_ratios={} 
for brands in comb:
    #print brands
    lift_val=lift(brands[0],brands[1])
    lift_ratios[brands]=lift_val
lift_ratios  #dictionary of {(brand1, brand2):lift value}

{('acura', 'audi'): 2.1309489961341814,
 ('acura', 'ford'): 1.646951951951952,
 ('acura', 'infiniti'): 3.0086195286195285,
 ('acura', 'nissan'): 1.871830763147489,
 ('acura', 'subaru'): 1.8834229175023398,
 ('audi', 'ford'): 1.8058683683683683,
 ('audi', 'infiniti'): 2.5353535353535355,
 ('audi', 'nissan'): 1.4872269551984854,
 ('audi', 'subaru'): 1.7893921161069175,
 ('bmw', 'acura'): 1.4208487158112024,
 ('bmw', 'audi'): 1.6458511517461143,
 ('bmw', 'ford'): 1.4319200631499667,
 ('bmw', 'honda'): 1.2637856440048396,
 ('bmw', 'infiniti'): 1.8954009548864854,
 ('bmw', 'nissan'): 1.1890698126809396,
 ('bmw', 'pontiac'): 1.4189319518000125,
 ('bmw', 'subaru'): 1.1839027089355016,
 ('bmw', 'toyota'): 1.2850237366687791,
 ('ford', 'subaru'): 1.8306420138550104,
 ('honda', 'acura'): 2.4167648175261647,
 ('honda', 'audi'): 1.351817343764635,
 ('honda', 'ford'): 2.0559930354952316,
 ('honda', 'infiniti'): 1.4532809796352988,
 ('honda', 'nissan'): 2.4214216117922294,
 ('honda', 'subaru'): 1.37

#### Task B:
What insights can you offer brand managers from your analysis in Task A    (choose two brands that you can offer the most interesting/useful insights for)? 

#### Task C:
What are 5   most frequently mentioned attributes of cars in the discussions? Note that the same attribute may be described by different words –   e.g., pick-up and acceleration may both referto a more general attribute, “performance”. You have to make suitable replacements. Now pick the 5 most frequently mentioned brands. Which attributes are most strongly associated with which of these 5   brands? You DON’T have to do a sentiment analysis for this assignment.
While BMW has claimed that they are the “ultimate driving machine”, is    that how people feel on Edmunds? Show your analysis.   

In [142]:
top5brands = top10[:5]
top5brands

import nltk
from nltk.corpus import wordnet
from nltk import pos_tag, word_tokenize
message_list=forum_data['message_without_stopwords'].values.tolist()
message_list
message_list[0] #the first job description (complete)
wordsinlist=[]
for i in range(0,len(message_list)):
    words1=word_tokenize(message_list[i])
    wordsinlist.append(words1)
wordsinlist[:10]

# wordnet.synsets("comfort")

[['entry',
  'level',
  'performance',
  'luxury',
  'sedans',
  'hot',
  'area',
  'car',
  'manufacturers',
  'enthusiasts',
  'alike',
  '.',
  'sedans',
  'show',
  'performance',
  'fun',
  'drive',
  'handling',
  'well',
  '.',
  'show',
  'luxury',
  'offering',
  'leather',
  'important',
  'creatures',
  'toyota',
  'found',
  'luxurious',
  'upscale',
  'sedans',
  '.',
  'selection',
  'vehicles',
  'compete',
  'space',
  'vast',
  'varied.the',
  'benchmarks',
  'define',
  'bmw',
  '330',
  'audi',
  'audi',
  '3.0.',
  'many',
  'contenders',
  ',',
  'acura',
  'tl-s',
  ',',
  'chryslerm',
  ',',
  'jaguar',
  'x-type',
  ',',
  'infiniti',
  'i35',
  ',',
  'mercedes',
  'mercedes',
  ',',
  'volvo',
  'volvo',
  't5',
  '.',
  'companies',
  'contenders',
  'emphasize',
  'performance',
  'lexus',
  'is300',
  'luxury',
  'lexus',
  'es300',
  '.',
  'field',
  'continously',
  'growing',
  ',',
  'recent',
  'additions',
  'cadillac',
  'cts',
  'infiniti',
  'pont

In [143]:
from nltk.probability import FreqDist
words = [item for sublist in wordsinlist for item in sublist]

words[:10]

['entry',
 'level',
 'performance',
 'luxury',
 'sedans',
 'hot',
 'area',
 'car',
 'manufacturers',
 'enthusiasts']

In [144]:
wordsfreq = dict((x,words.count(x)) for x in set(words))
sorted(((wordsfreq[x], x) for x in wordsfreq), reverse=True)

[(25349, '.'),
 (22309, ','),
 (4578, "'s"),
 (4372, 'car'),
 (4040, "n't"),
 (3805, ')'),
 (3641, '?'),
 (3599, 'bmw'),
 (3579, '``'),
 (3363, '('),
 (2955, 'i'),
 (2655, 'it'),
 (2473, '...'),
 (2436, "''"),
 (2153, ':'),
 (2143, 'cars'),
 (2069, 'like'),
 (1891, 'would'),
 (1882, 'tl'),
 (1764, 'one'),
 (1702, '-'),
 (1633, '!'),
 (1518, 'do'),
 (1487, 'get'),
 (1458, '$'),
 (1433, 'pontiac5'),
 (1393, 'think'),
 (1107, 'drive'),
 (1098, 'better'),
 (1083, 'performance'),
 (1060, 'that'),
 (1058, "'m"),
 (1042, 'much'),
 (1034, 'new'),
 (1011, 'even'),
 (1010, '3'),
 (1002, 'audi'),
 (974, 'people'),
 (943, 'you'),
 (933, 'acura'),
 (928, 'driving'),
 (920, '&'),
 (893, 'sedan'),
 (892, 'good'),
 (856, 'really'),
 (842, 'know'),
 (838, 'lexus'),
 (834, 'luxury'),
 (820, 'want'),
 (818, 'well'),
 (773, 'also'),
 (735, 'still'),
 (727, ';'),
 (726, 'best'),
 (721, 'say'),
 (714, 'engine'),
 (712, 'rwd'),
 (704, '330i'),
 (696, 'honda'),
 (686, 'could'),
 (683, 'series'),
 (675, "'ve")

In [145]:
##categories?? acceleration(pick-up,weight, w8, rpm,speed,mph), exterior(spoiler, carbon,lift,paint,color,finish,aluminum)
#              price(cost,affordable,expensive,cheap,value,fuel,efficiency,mpg,hybrid), 
#              handling(suspension,turning,cornering,steering,awd,rwd)
#              power(torque,hp,horsepower,engine size, engine,automatic,manual,transmission), make(sedan,coupe,convertible)
#              interior(luxury,features, stereo,gps,radio,cd player, heated, boot, clubs,ergonomics,nav,leather)
#              quality(reliability,build,safety, crash), 
print words[:10]
frq=FreqDist(words)
#frq
frq.most_common(50)
count=0
for w in words:
    if w =='build':
        count+=1
count

['entry', 'level', 'performance', 'luxury', 'sedans', 'hot', 'area', 'car', 'manufacturers', 'enthusiasts']


86

In [153]:
attribute_dict = {'w8':'acceleration','weight':'acceleration','pick-up':'acceleration','rpm':'acceleration','speed':'acceleration','mph':'acceleration',
                  'cost':'price','affordable':'price','expensive':'price','cheap':'price','value':'price','fuel':'price','efficiency':'price','mpg':'price','hybrid':'price',
                  'suspension':'handling','turning':'handling','cornering':'handling','steering':'handling','awd':'handling','rwd':'handling',
                  'torque':'power','hp':'power','horsepower':'power','engine':'power','automatic':'power','manual':'power','transmission':'power',
                  'luxury':'interior','features':'interior','stereo':'interior','gps':'interior','cd':'interior','heated':'interior','boot':'interior','clubs':'interior','nav':'interior','leather':'interior',
                  'spoiler':'exterior','carbon':'exterior','lift':'exterior','paint':'exterior','color':'exterior','finish':'exterior','aluminum':'exterior',
                  'reliability':'quality','build':'quality','safety':'quality','crash':'quality'}

#replaces words that fall into attribute category with that attribute
def replace_attribute(s):
    attr=[]
    for word in s:
        if word in attribute_dict:
            attr.append(attribute_dict[word])
        else:
            attr.append(word)    
    return attr

counter = 0
new_words = replace_attribute(words)
#print new_words
for w in new_words:
    if w =='acceleration':
        counter+=1
counter

1115

In [156]:
attribute_list = ['acceleration', 'handling','power', 'price','interior','exterior','quality']
##need help on brainstorming other attributes, i don't know cars
handlecount = 0
powcount=0
pricecount=0
accelcount=0
makecount = 0 
intcount=0
extcount=0
qualitycount=0
for w in new_words:
    if w =='acceleration':
        accelcount+=1
    if w=='handling':
        handlecount+=1
    if w=='power':
        powcount+=1
    if w=='price':
        pricecount+=1
    if w=='interior':
        intcount+=1
    if w=='exterior':
        extcount+=1
    if w=='quality':
        qualitycount+=1
attr_freq_list = [['acceleration',accelcount],['handling',handlecount],['price',pricecount],['power',powcount],['interior',intcount],['exterior',extcount],['quality',qualitycount]]
attr_freq_list.sort(key=lambda x: x[1], reverse=True)

attr_freq_list[:5]


[['power', 3401],
 ['handling', 2389],
 ['interior', 2087],
 ['price', 1915],
 ['acceleration', 1115]]

In [148]:
def lift(a, b):
    num = len(forum_data)
    num_a = len(forum_data[forum_data['message_without_stopwords'].str.contains(a)])
    num_b = len(forum_data[forum_data['message_without_stopwords'].str.contains(b)])
    if_has_a = forum_data[forum_data['message_without_stopwords'].str.contains(a)]
    num_a_and_b = len(if_has_a['message_without_stopwords'][if_has_a['message_without_stopwords'].str.contains(b)])
    return num*float(num_a_and_b)/(num_a * num_b)

In [157]:
temp_list = ['power','handling','interior','price','acceleration']

listof5 = listof10[:5]

attribute_df = pd.DataFrame(columns=listof5, index=temp_list)

for attribute, series in list(attribute_df.iterrows()):
    for brand in series.index:
        attribute_df[brand].loc[attribute] = lift(brand, attribute)

attribute_df

,bmw,toyota,pontiac,honda,acura
power,1.49637,1.53075,1.59265,1.8009,1.75574
handling,1.55007,1.52657,1.67789,1.45791,1.61568
interior,1.35052,1.36037,1.92626,1.31775,1.65718
price,1.54689,1.44077,1.71213,1.5234,1.49173
acceleration,1.46756,1.83466,1.76869,1.78801,1.53508


#### Task D:
What advice will you give to a (i) product manager, and (ii) marketing/advertising manager of these brands based on your analysis in Task C? For this assignment, you can assume the sentiment (e.g., that it   is    positive). 

#### Task E:
Which is the most aspirational brand in your data in terms of people actually wanting to buy or own? Describe your analysis. What are the business implications for this brand? Provide the following details in your write-up (I    am not setting a   strict page limit, but 3-4 pages, single-spaced, 11 font size should be enough with screenshots)

1.Which forum you chose (provide URL) 

2.Which 10 brands you chose –   provide the frequency table

3.Show all lift calculations in a   table.

4.Show the MDS map (put screenshots in your report)

5.State the 5   attributes you chose (again, a   table is    good here). 

6.For task E, provide all details of your analysis –   e.g., how you measured “aspirational” and howyou found the most aspirational brand.

7.Advice/insights based on your analysis for brand, product and advertising managers. Your submission should include all scripts as well as your answers to the questions above (generally speaking, I  won’t run these scripts, but if   the numbers don’t look right, I may run some of them). Also include the final data file after all replacements of models and attributes have been done.